In [11]:
import re
import nrrd
import os, glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from patchify import patchify, unpatchify
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

import loss
import Helper
import allMetrics

os.environ["CUDA_VISIBLE_DEVICES"]="1"

data_dir = '/media/dro/JHSeagate/FYP/jh_fyp_work/ct_scans_data/raw_data/'

train_image = os.path.join(data_dir, 'train_data/training_images/')
valid_image = os.path.join(data_dir, 'valid_data/valid_images/')
test_image = os.path.join(data_dir, 'test_data/testing_images/')

train_mask = os.path.join(data_dir, 'train_data/training_masks/')
valid_mask = os.path.join(data_dir, 'valid_data/valid_masks/')
test_mask = os.path.join(data_dir, 'test_data/testing_masks/')

train_image_path = [train_image + file_name for file_name in os.listdir(train_image)]
valid_image_path = [valid_image + file_name for file_name in os.listdir(valid_image)]
test_image_path = [test_image + file_name for file_name in os.listdir(test_image)]

train_mask_path = [train_mask + file_name for file_name in os.listdir(train_mask)]
valid_mask_path = [valid_mask + file_name for file_name in os.listdir(valid_mask)]
test_mask_path = [test_mask + file_name for file_name in os.listdir(test_mask)]



model_checkpoint_path = '/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/Final/SavedModels/resampled_5mm.hdf5'

model = tf.keras.models.load_model(model_checkpoint_path, custom_objects = {                                                                
                                                                                'dice_coef':allMetrics.dice_coef,
                                                                                   'tversky_crossentropy':loss.tversky_crossentropy})
input_shape = (64,64,64)
num_class = 1
desired_step = 32

test_patient_img = Helper.process_scan(test_image_path[5])
test_patient_msk = Helper.read_nrrd_file(test_mask_path[5])

test_patch = patchify(test_patient_img, input_shape, step = desired_step) # Yield 15 x 15 x 3 of size 64 x 64 x 32 cubes
test_mask_patch = patchify(test_patient_msk, input_shape, step = desired_step)

test_patient_cubes = np.reshape(test_patch, (-1, test_patch.shape[3], test_patch.shape[4], test_patch.shape[5])) # Collapse 15 x 15 x 3 into 675 cubes x 64 x 64 x 32
test_patient_mask_cubes = np.reshape(test_mask_patch, (-1, test_mask_patch.shape[3], test_mask_patch.shape[4], test_mask_patch.shape[5])) # 675 x 64 x 64 x 32

test_patient_input = np.expand_dims(test_patient_cubes, axis=4)
test_patient_mask_input = np.expand_dims(test_patient_mask_cubes, axis=4)

test_patient_pred = model.predict(test_patient_input)

test_patient_pred_reshaped = np.reshape(test_patient_pred, 
                                        (test_patch.shape[0], test_patch.shape[1], test_patch.shape[2],
                                         test_patch.shape[3], test_patch.shape[4], test_patch.shape[5]))

reconstruct_pred = unpatchify(test_patient_pred_reshaped, test_patient_img.shape)
reconstruct_pred[reconstruct_pred < 1] = 0

@interact
def explore_prediction(layer = (0,95), view = ["axial"]):
    if view == 'axial':
        img_array_view = test_patient_img[:, :, layer]
        msk_array_view = test_patient_msk[:, :, layer]
        pred_array_view = reconstruct_pred[:, :, layer]
# , cmap='gray'
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(img_array_view, cmap = "gray", aspect = "auto")
    plt.title('Img', fontsize=10)
    plt.subplot(1, 3, 2)
    plt.imshow(msk_array_view, cmap = "gray",aspect = "auto")
    plt.title('Mask', fontsize=10)
    plt.subplot(1, 3, 3)
    plt.imshow(pred_array_view, cmap = "gray",aspect = "auto")
    plt.title('Prediction', fontsize=10)
    plt.axis('off')
    plt.show()
    
headerInfo = nrrd.read_header(test_image_path[5])

modelname = 'P276_resampled_5mm_Model'
pred_volume_file_location = f"/media/dro/JHSeagate/FYP/jh_fyp_work/ct_scans_data/prediction_data/resampled_5mm_Predictions/{modelname}.nrrd"
nrrd.write(pred_volume_file_location, reconstruct_pred, headerInfo)

interactive(children=(IntSlider(value=47, description='layer', max=95), Dropdown(description='view', options=(…

In [2]:
def generate_reconstruct_pred(patient_path):
    
    headerInfo = nrrd.read_header(patient_path)
    
    patient_idx = int(re.findall(r'\d+', patient_path[-23:-1])[0])
    pred_volume_file_location = f"/media/dro/JHSeagate/FYP/jh_fyp_work/ct_scans_data/prediction_data/resampled_5mm_Predictions/P{patient_idx}_pred.nrrd"

    
    input_shape = (64,64,64)
    num_class = 1
    desired_step = 32
    
    test_patient_img = Helper.process_scan(patient_path)
    test_patient_msk = Helper.read_nrrd_file(patient_path)

    test_patch = patchify(test_patient_img, input_shape, step = desired_step) # Yield 15 x 15 x 3 of size 64 x 64 x 32 cubes
    test_mask_patch = patchify(test_patient_msk, input_shape, step = desired_step)

    test_patient_cubes = np.reshape(test_patch, (-1, test_patch.shape[3], test_patch.shape[4], test_patch.shape[5])) # Collapse 15 x 15 x 3 into 675 cubes x 64 x 64 x 32
    test_patient_mask_cubes = np.reshape(test_mask_patch, (-1, test_mask_patch.shape[3], test_mask_patch.shape[4], test_mask_patch.shape[5])) # 675 x 64 x 64 x 32

    test_patient_input = np.expand_dims(test_patient_cubes, axis=4)
    test_patient_mask_input = np.expand_dims(test_patient_mask_cubes, axis=4)

    test_patient_pred = model.predict(test_patient_input)

    test_patient_pred_reshaped = np.reshape(test_patient_pred, 
                                            (test_patch.shape[0], test_patch.shape[1], test_patch.shape[2],
                                             test_patch.shape[3], test_patch.shape[4], test_patch.shape[5]))

    reconstruct_pred = unpatchify(test_patient_pred_reshaped, test_patient_img.shape)
    nrrd.write(pred_volume_file_location, reconstruct_pred, headerInfo)
    
    return

In [3]:
for file_path in test_path:
    generate_reconstruct_pred(file_path)

NameError: name 'test_path' is not defined